In [ ]:
import requests
import json
import pandas as pd
import os

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

RECOMENDATION_PROMPT = """
Ты — очень опытный IT-рекрутер. На основе предоставленного текста резюме, дай 5-6 чётких, полезных рекомендаций по улучшению текста резюме, чтобы соискатель выглядел более привлекательным для работодателей в сфере IT.
Не проси убрать лишнюю информация если это: водительские права, семейное положение, гражданство. В нашей стране - это стандарт 
Формат ответа: только JSON. Не добавляй никаких пояснений, комментариев или текста вне JSON.

Ответ должен быть строго в виде:

{
  "recommendations": [
    "рекомендация..."
    "рекомендация...",
    ...
  ]
}
"""

def prepare_prompt_recomendations(cv_text: str) -> list:
    return [
        {"role": "system", "content": RECOMENDATION_PROMPT},
        {"role": "user", "content": f"Вот текст резюме:\n{cv_text}"}
    ]

def query_mistral(prompt: list):
    response = requests.post(
        API_URL,
        headers=HEADERS,
        json={
            "model": MODEL_NAME,
            "messages": prompt,
            "temperature": 0.3,
            "max_tokens": 1500
        },
        timeout=120
    )
    response.raise_for_status()
    content = response.json()["choices"][0]["message"]["content"]
    content = content.replace("```json", "").replace("```", "").strip()

    if content.lower() == "null":
        return None

    return json.loads(content)

def process_single_resume_recomendations(input_path, output_path, text_column="CV_Text"):
    ext = os.path.splitext(input_path)[-1].lower()
    if ext == ".csv":
        df = pd.read_csv(input_path)
    elif ext in [".xls", ".xlsx"]:
        df = pd.read_excel(input_path)
    else:
        raise ValueError("Поддерживаются только CSV и Excel файлы")

    if text_column not in df.columns:
        raise ValueError(f"В файле нет колонки '{text_column}'")

    cv_text = df[text_column].iloc[0]  # берём первую строку
    prompt = prepare_prompt_recomendations(cv_text)
    result = query_mistral(prompt)

    if result is None:
        print("Модель вернула null")
        result = {"recommendations": []}


    

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    print(f"✅ Рекомендации сохранены в: {output_path}")


In [2]:
process_single_resume_recomendations(
    input_path="Inference_cv/extracted_cv_text.xlsx",   # файл с одним резюме
    output_path="recomendations.json"   # 
)

✅ Рекомендации сохранены в: recomendations.json
